### Small Tensorflow project using MNIST dataset.

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanAbsoluteError, MeanSquaredError
from tensorflow.keras.optimizers import Adam, SGD, Adagrad
from tensorflow.keras.metrics import Mean, SparseCategoricalAccuracy

##### Load MNIST dataset

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Rescale to range 0..1
x_train = x_train / 255.0
x_test = x_test / 255.0

# Add channels
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

##### Shuffle dataset and prepare batches

In [5]:
training_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
training_dataset = training_dataset.shuffle(buffer_size=training_dataset.cardinality()).batch(32)

testing_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

##### Create model class

In [6]:
class model_1conv(Model):
    # Constructor
    def __init__(self, n_of_filters, n_of_dense_neurons):
        super(model_1conv, self).__init__()

        self.conv = Conv2D(filters=n_of_filters,
                           kernel_size=3,
                           activation="relu")
        self.flatten = Flatten()
        self.dense_1 = Dense(n_of_dense_neurons, activation="relu")
        self.dense_2 = Dense(10)

    # Call method
    def call(self, x):
        x = self.conv(x)
        x = self.flatten(x)
        x = self.dense_1(x)
        x = self.dense_2(x)

        return x

In [10]:
class model_2conv(Model):
    # Constructor
    def __init__(self, n_of_filters, n_of_dense_neurons):
        super(model_2conv, self).__init__()

        self.conv_1 = Conv2D(filters=n_of_filters[0],
                             kernel_size=3,
                             activation="relu")
        self.conv_1 = Conv2D(filters=n_of_filters[1],
                             kernel_size=3,
                             activation="relu")
        self.flatten = Flatten()
        self.dense_1 = Dense(n_of_dense_neurons, activation="relu")
        self.dense_2 = Dense(10)

    # Call method
    def call(self, x):
        x = self.conv_1(x)
        x = self.conv_2(x)
        x = self.flatten(x)
        x = self.dense_1(x)
        x = self.dense_2(x)

        return x

##### Create instances of models

In [11]:
# Different model parameters
filters = [4, 8, 16, 32, 64]
dense_neurons = [16, 32, 64, 128, 256]

# Models with one Convolutional Layer
conv1_var_f = [model_1conv(n_of_filters=f, n_of_dense_neurons=128) for f in filters]            # Variated filters
conv1_var_d = [model_1conv(n_of_filters=32, n_of_dense_neurons=dn) for dn in dense_neurons]     # Variated number of neurons in Dense layer

# Model with two Convolutional Layers
conv2_var_f = [model_2conv(n_of_filters=(f, f), n_of_dense_neurons=128) for f in filters]               # Variated filters
conv2_var_f = [model_2conv(n_of_filters=(32, 32), n_of_dense_neurons=dn) for dn in dense_neurons]       # Variated number of neurons in Dense layer

##### Loss functions and optimizers for training

In [13]:
loss_objects = [MeanAbsoluteError, MeanSquaredError, SparseCategoricalCrossentropy]
optimizers = [Adam, SGD, Adagrad]

In [14]:
# Metrics for measurement of model's loss and accuracy
training_loss = Mean(name="train_loss")
testing_loss = Mean(name="test_loss")

training_accuracy = SparseCategoricalAccuracy(name="train_acc")
testing_accuracy = SparseCategoricalAccuracy(name="test_acc")

##### Model's training and testing functions

In [15]:
# Define function as Tensorflow Graph
@tf.function
def train_step(model, loss_object, optimizer, images, labels):

    # Automatic differentiation
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    training_loss(loss)
    training_accuracy(labels, predictions)

In [16]:
# Define function as Tensorflow Graph
@tf.function
def test_step(model, loss_object, images, labels):
    predictions = model(images, training=False)
    loss = loss_object(labels, predictions)

    testing_loss(loss)
    testing_accuracy(labels, predictions)

##### Training and model metrics acquisition

In [ ]:
N_OF_EPOCHS = 5

for epoch in N_OF_EPOCHS:
    raise NotImplementedError